### establishment opening hours dataframe

### Creating grab_df and subsetting opening hours column

In [5]:
import pandas as pd

# import grab dataset
grab_df = pd.read_csv("data/Grab SG Restaurants.csv")

# subset openinghour df with records being dictionaries
op_hours_dict_df = grab_df[["opening_hours"]]

op_hours_dict_df.head()

,opening_hours
0,"{""open"": true, ""displayedHours"": ""10:30-21:30""..."
1,"{""open"": true, ""displayedHours"": ""12:00-19:00""..."
2,"{""open"": true, ""displayedHours"": ""11:30-20:45""..."
3,"{""open"": true, ""displayedHours"": ""11:00-21:00""..."
4,"{""open"": true, ""displayedHours"": ""10:30-21:00""..."


#### Cleaning the column of dictionaries

In [6]:
# replacing " " in the dictionary to ' '
op_hours_dict_df = op_hours_dict_df['opening_hours'].str.replace('"',"'")
op_hours_dict_df = pd.DataFrame(op_hours_dict_df)

# replacing 'true' to 'True'
op_hours_dict_df = op_hours_dict_df['opening_hours'].str.replace('true',"'True'")
op_hours_dict_df = pd.DataFrame(op_hours_dict_df)

print(op_hours_dict_df)
print(type(op_hours_dict_df))

                                           opening_hours
0      {'open': 'True', 'displayedHours': '10:30-21:3...
1      {'open': 'True', 'displayedHours': '12:00-19:0...
2      {'open': 'True', 'displayedHours': '11:30-20:4...
3      {'open': 'True', 'displayedHours': '11:00-21:0...
4      {'open': 'True', 'displayedHours': '10:30-21:0...
...                                                  ...
16131  {'displayedHours': 'Closed', 'sun': 'Closed', ...
16132  {'displayedHours': 'Closed', 'sun': '12:00-14:...
16133  {'displayedHours': 'Closed', 'sun': '11:00-22:...
16134  {'displayedHours': 'Closed', 'sun': '09:00-22:...
16135  {'displayedHours': 'Closed', 'sun': 'Closed', ...

[16136 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


#### Splitting the dictionaries into separate columns in a dataframe

In [7]:
import ast
from pandas import json_normalize

# Ensure that the column is in a proper list of dictionaries format
op_hours_dict_df['opening_hours'] = op_hours_dict_df['opening_hours'].apply(ast.literal_eval)

# Use json_normalize to convert the dictionaries into separate columns
op_hours_dict_df_expanded = op_hours_dict_df.join(json_normalize(op_hours_dict_df['opening_hours'])).drop('opening_hours', axis=1)

print(op_hours_dict_df_expanded)


       open displayedHours                      sun                      mon  \
0      True    10:30-21:30              10:30-21:30              10:30-21:15   
1      True    12:00-19:00              12:00-19:00              12:00-19:00   
2      True    11:30-20:45              11:30-20:45              11:30-20:45   
3      True    11:00-21:00              11:00-21:00              11:00-21:00   
4      True    10:30-21:00              11:00-21:30              11:00-21:30   
...     ...            ...                      ...                      ...   
16131   NaN         Closed                   Closed              07:00-13:00   
16132   NaN         Closed  12:00-14:00 17:30-21:30  12:00-14:00 17:30-21:30   
16133   NaN         Closed              11:00-22:15  11:00-14:45 18:00-22:15   
16134   NaN         Closed              09:00-22:00              06:30-22:00   
16135   NaN         Closed                   Closed              07:00-15:00   

                           tue         

#### Joining id_source column from grab_df to op_hours_dict_df_expanded

In [8]:
# Joining id_source column from grab_df to op_hours_dict_df_expanded
openinghour_df = pd.concat([grab_df[["id_source"]], op_hours_dict_df_expanded], axis=1, sort=False)
                  
print(openinghour_df)

              id_source  open displayedHours                      sun  \
0             SGDD01085  True    10:30-21:30              10:30-21:30   
1      4-CYTDLPUJEP53N6  True    12:00-19:00              12:00-19:00   
2             SGDD05605  True    11:30-20:45              11:30-20:45   
3             SGDD07548  True    11:00-21:00              11:00-21:00   
4      4-CY2ZLZLTT2EDJ2  True    10:30-21:00              11:00-21:30   
...                 ...   ...            ...                      ...   
16131  4-CZNFTYKKVA3UN2   NaN         Closed                   Closed   
16132         SGDD01192   NaN         Closed  12:00-14:00 17:30-21:30   
16133         SGDD00961   NaN         Closed              11:00-22:15   
16134  4-C2AHJVATN7C2RE   NaN         Closed              09:00-22:00   
16135  4-C2DXFEK2R6KJTJ   NaN         Closed                   Closed   

                           mon                      tue  \
0                  10:30-21:15              10:30-21:15   
1    

#### Adding index column with prefix

In [9]:
# Create a new index column and add a prefix
openinghour_df['openinghour_id'] = 'op_' + (openinghour_df.index + 1).astype(str)

# Set new index column to be the first column
openinghour_df = openinghour_df.set_index('openinghour_id').reset_index()

print(openinghour_df)

      openinghour_id         id_source  open displayedHours  \
0               op_1         SGDD01085  True    10:30-21:30   
1               op_2  4-CYTDLPUJEP53N6  True    12:00-19:00   
2               op_3         SGDD05605  True    11:30-20:45   
3               op_4         SGDD07548  True    11:00-21:00   
4               op_5  4-CY2ZLZLTT2EDJ2  True    10:30-21:00   
...              ...               ...   ...            ...   
16131       op_16132  4-CZNFTYKKVA3UN2   NaN         Closed   
16132       op_16133         SGDD01192   NaN         Closed   
16133       op_16134         SGDD00961   NaN         Closed   
16134       op_16135  4-C2AHJVATN7C2RE   NaN         Closed   
16135       op_16136  4-C2DXFEK2R6KJTJ   NaN         Closed   

                           sun                      mon  \
0                  10:30-21:30              10:30-21:15   
1                  12:00-19:00              12:00-19:00   
2                  11:30-20:45              11:30-20:45   
3      

In [10]:
import numpy as np

# Count no. of NaN in dataframe
nan_count = openinghour_df.isna().sum().sum()

print(f'Total number of NaN values in the DataFrame: {nan_count}')

Total number of NaN values in the DataFrame: 21830


In [11]:
# Number of NaN values for each column
nan_count_per_column_df = openinghour_df.isna().sum()

print(nan_count_per_column_df)

openinghour_id        0
id_source             0
open               8981
displayedHours        0
sun                   0
mon                   0
tue                   0
wed                   0
thu                   0
fri                   0
sat                   0
tempClosed        12849
dtype: int64


In [12]:
# No. of establishment minus no. of tempClosed equals 'NaN'
count_open = 16136-12849
print(count_open)

3287


In [13]:
# Create a new DataFrame based on the condition Open=NaN and tempClosed=True
opennull_tempclosetrue = openinghour_df[pd.isna(openinghour_df['open']) & (openinghour_df['tempClosed'] == 'True')]

print('Total no. of rows of tempClosed establishment = ' + str(len(opennull_tempclosetrue)))

Total no. of rows of tempClosed establishment = 3287


In [14]:
# Create a new DataFrame based on the condition tempClosed=True
tempclosedtrue = openinghour_df[pd.isna(openinghour_df['tempClosed'])]

print('Total no. of rows of (tempClosed is NaN) establishment = ' + str(len(tempclosedtrue)))

Total no. of rows of (tempClosed is NaN) establishment = 12849


In [15]:
# Create a new DataFrame based on the condition open=NaN
opentrue = openinghour_df[(openinghour_df['open'] == 'True')]

print('Total no. of rows of (open == "True") establishment = ' + str(len(opentrue)))

Total no. of rows of (open == "True") establishment = 7155


### There are 12849 - 7155 = 5694 establishment with  open==NaN and tempClosed==NaN

In [16]:
12849-7155

5694

In [17]:
# Create a new DataFrame based on the condition open=NaN and tempClosed=NaN
opennull_tempClosednull = openinghour_df[pd.isna(openinghour_df['open']) & pd.isna(openinghour_df['tempClosed'])]

print('Total no. of rows of (open == "NaN" and tempClosed == "NaN") establishment = ' + str(len(opennull_tempClosednull)))
print(opennull_tempClosednull.head())
print(type(opennull_tempClosednull))

Total no. of rows of (open == "NaN" and tempClosed == "NaN") establishment = 5694
    openinghour_id         id_source open displayedHours  \
351         op_352  4-CY22C4LVJNAJL2  NaN         Closed   
363         op_364  4-CY4GN2MGHF3JCA  NaN         Closed   
364         op_365  4-CZDVVZKBRTVTBE  NaN         Closed   
371         op_372  4-CZKER762C62GRN  NaN         Closed   
372         op_373         SGDD02751  NaN         Closed   

                         sun                      mon  \
351  11:00-16:55 17:00-20:45                   Closed   
363              11:30-20:00              11:30-20:00   
364  12:00-14:30 17:00-20:15  12:00-14:30 17:00-20:15   
371                   Closed              10:00-13:00   
372              10:00-20:00              10:00-20:00   

                         tue                      wed  \
351                   Closed                   Closed   
363              11:30-20:00              11:30-20:00   
364  12:00-14:30 17:00-20:15  12:00-14:30 1

In [18]:
#Subsetting grab_df with address and id_source
grab_df[["address","id_source"]].head()

,address,id_source
0,Texas Chicken - VivoCity,SGDD01085
1,Subway - Vivocity,4-CYTDLPUJEP53N6
2,LiHO Tea - Vivo City,SGDD05605
3,Wingstop - VivoCity,SGDD07548
4,Le Shrimp Ramen - Vivocity,4-CY2ZLZLTT2EDJ2


In [19]:
# Performing an inner join on the 'id_source' column to include 'address' column to opennull_tempClosednull
#syntax for .merge >> inner_joined_df = df1.merge(df2, on='key', how='inner')

opennull_tempClosednull_address = grab_df[["address","id_source"]].merge(opennull_tempClosednull, on='id_source', how='inner')
opennull_tempClosednull_address.head()

,address,id_source,openinghour_id,open,displayedHours,sun,mon,tue,wed,thu,fri,sat,tempClosed
0,Alijiang 阿里疆- Vivocity,4-CY22C4LVJNAJL2,op_352,NaN,Closed,11:00-16:55 17:00-20:45,Closed,Closed,Closed,Closed,Closed,Closed,NaN
1,Bottles & Bottles - Vivocity,4-CY4GN2MGHF3JCA,op_364,NaN,Closed,11:30-20:00,11:30-20:00,11:30-20:00,11:30-20:00,11:30-20:00,11:30-20:00,11:30-20:00,NaN
2,Royal Taj - Mess Hall - Village Hotel,4-CZDVVZKBRTVTBE,op_365,NaN,Closed,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,12:00-14:30 17:00-20:15,NaN
3,Kheng Nam Lee Curry Rice - Seah Im Food Centre,4-CZKER762C62GRN,op_372,NaN,Closed,Closed,10:00-13:00,10:00-13:00,10:00-13:00,10:00-13:00,10:00-13:00,10:00-13:00,NaN
4,A-One Claypot House - VivoCity,SGDD02751,op_373,NaN,Closed,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,10:00-20:00,NaN


### Based checking on first 5 establishments at time of interim project, they are currently opened, meaning they were not permanently closed at time of dataset. Therefore, for sake of analysis in our project, it is assumed that all entries with open=NaN and tempClosed=NaN are not closed permanently and therefore included in the analysis of the dataset.

#### Replacing open=NaN to True and tempClosed=NaN to False for rows with open=NaN & tempClosed=NaN

In [20]:
# Conditionally fill NaN values in open with 'True' and in tempClosed with 'False'
openinghour_df.loc[openinghour_df['open'].isna() & openinghour_df['tempClosed'].isna(), ['open', 'tempClosed']] = [True, False]

#to check filling of NaN values for open=NaN and tempClosed=NaN 
openinghour_df.iloc[351]

openinghour_id                     op_352
id_source                4-CY22C4LVJNAJL2
open                                 True
displayedHours                     Closed
sun               11:00-16:55 17:00-20:45
mon                                Closed
tue                                Closed
wed                                Closed
thu                                Closed
fri                                Closed
sat                                Closed
tempClosed                          False
Name: 351, dtype: object

#### For open=true and tempClosed=NaN, replace tempClosed=NaN to False
#### For open=NaN and tempClosed=true, replace open=NaN to False
#### For any of the days not = 'Closed', replace open to True and tempClosed to False

In [21]:
# Conditionally fill NaN values in tempClosed with 'False'
openinghour_df.loc[openinghour_df['tempClosed'].isna(), ['tempClosed']] = ['False']

# Conditionally fill NaN values in open with 'False'
openinghour_df.loc[openinghour_df['open'].isna(), ['open']] = ['False']

# Update 'open' to True and 'tempClosed' to False where any of the days are not = 'Closed'
openinghour_df.loc[(openinghour_df['displayedHours'] != 'closed') 
                     | (openinghour_df['sun'] != 'Closed') 
                     | (openinghour_df['mon'] != 'Closed')
                     | (openinghour_df['tue'] != 'Closed') 
                     | (openinghour_df['wed'] != 'Closed')
                     | (openinghour_df['thu'] != 'Closed') 
                     | (openinghour_df['fri'] != 'Closed')
                     | (openinghour_df['sat'] != 'Closed') 
                     , ['open', 'tempClosed']] = [True, False]

openinghour_df.tail()

,openinghour_id,id_source,open,displayedHours,sun,mon,tue,wed,thu,fri,sat,tempClosed
16131,op_16132,4-CZNFTYKKVA3UN2,True,Closed,Closed,07:00-13:00,Closed,Closed,Closed,Closed,Closed,False
16132,op_16133,SGDD01192,True,Closed,12:00-14:00 17:30-21:30,12:00-14:00 17:30-21:30,12:00-14:00 17:30-21:30,12:00-14:00 17:30-21:30,12:00-14:00 17:30-21:30,12:00-14:00,12:00-14:00,False
16133,op_16134,SGDD00961,True,Closed,11:00-22:15,11:00-14:45 18:00-22:15,Closed,11:00-14:45 18:00-22:15,11:00-14:45 18:00-22:15,11:00-22:45,11:00-22:45,False
16134,op_16135,4-C2AHJVATN7C2RE,True,Closed,09:00-22:00,06:30-22:00,06:30-22:00,06:30-22:00,06:30-22:00,06:30-22:00,06:30-22:00,False
16135,op_16136,4-C2DXFEK2R6KJTJ,True,Closed,Closed,07:00-15:00,07:00-15:00,07:00-15:00,07:00-15:00,07:00-15:00,07:00-15:00,False


#### 9 establishment where displayedHours and all days are = 'Closed'

# should we drop this 9 rows? some are mix n match, some arenot closed on google. MAYBE, they not using grab anymore?

In [22]:
#displaying rows where displayedHours and all days are = 'Closed'
all_closed = openinghour_df[(openinghour_df['displayedHours'] == 'Closed')
                             & (openinghour_df['sun'] == 'Closed') 
                             & (openinghour_df['mon'] == 'Closed')
                             & (openinghour_df['tue'] == 'Closed') 
                             & (openinghour_df['wed'] == 'Closed')
                             & (openinghour_df['thu'] == 'Closed') 
                             & (openinghour_df['fri'] == 'Closed')
                             & (openinghour_df['sat'] == 'Closed')] 
                     
# Update 'open' to False and 'tempClosed' to True where displayedHours and all days are = 'Closed'
print(len(all_closed))

all_closed_address = grab_df[["address","id_source"]].merge(all_closed, on='id_source', how='inner')
print(all_closed_address)


9
                                             address         id_source  \
0                 FOODQA-BJ_SG's Sea Resto GKMM Mart  4-CZEXEPEWN6KKEN   
1             Mix & Match Malls (Food) - Jurong East  4-C2CXV7CFKA4FWA   
2     Heng Kee Bak Kut Teh & Kway Chap - Tradehub 21  4-CY42TJB2CVEZEA   
3          Mix & Match Malls (Food) - Jurong Gateway  4-C2DTNY4JUETHLX   
4                     Monti A Casa - Botanic Gardens  4-CZDGPAEGT22ERT   
5  Jewel Changi Airport L1 and L2 - Mix & Match -...  4-CZJZV4JFC6EXRA   
6                      Changi Airport T3 Mix & Match  4-CZM1EUBUVUD3AX   
7  Jewel Changi Airport B1 and B2 - Mix & Match -...  4-C2CVJXAEC2B1EX   
8                     Sin Cuisine - Cheng San Centre  4-CZD1AP61NJWBJX   

  openinghour_id  open displayedHours     sun     mon     tue     wed     thu  \
0        op_1032  True         Closed  Closed  Closed  Closed  Closed  Closed   
1        op_4907  True         Closed  Closed  Closed  Closed  Closed  Closed   
2        op_51

### Saving dataframe to csv

In [23]:
#Saving new_df to csv
openinghour_df.to_csv('openinghour_df.csv', index = False)